In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import os
from dotenv import load_dotenv

In [3]:
# --- Determine Project Root and .env Path ---
# This logic assumes your project root is where the .git folder would be,
# and your .env file is directly in that root.
# For your structure: c:\Users\balde\SongRecommendation\

# Get the directory of the current notebook.
# If __file__ is defined (e.g. running as script), use its directory.
# Otherwise (interactive notebook), use current working directory.
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()

# Define project_root based on common structures.
# If notebook is in 'c:\Users\balde\SongRecommendation\SongRecommendation.ipynb'
# or 'c:\Users\balde\SongRecommendation\some_subfolder\SongRecommendation.ipynb'
# we want project_root to be 'c:\Users\balde\SongRecommendation\'
# A simple way is to assume the .env file is in the parent of the notebook's directory,
# or the current directory if the notebook is already in the root.

# Try to locate the .env file by checking common project root locations relative to the notebook
# This assumes your .env file is at c:\Users\balde\SongRecommendation\.env
project_root_candidate = os.path.abspath("c:/Users/balde/SongRecommendation") # Explicitly set your known project root

dotenv_path = os.path.join(project_root_candidate, '.env')

if not os.path.exists(dotenv_path):
    # Fallback if the explicit path is wrong or notebook is run from an unexpected CWD
    # Try current working directory
    dotenv_path_cwd = os.path.join(os.getcwd(), '.env')
    if os.path.exists(dotenv_path_cwd):
        dotenv_path = dotenv_path_cwd
        project_root_candidate = os.getcwd() # Update project_root if .env found in CWD
    else:
        # Try one level up from CWD (if notebook is in a subfolder of the project root)
        dotenv_path_parent = os.path.join(os.path.abspath(os.path.join(os.getcwd(), '..')), '.env')
        if os.path.exists(dotenv_path_parent):
            dotenv_path = dotenv_path_parent
            project_root_candidate = os.path.abspath(os.path.join(os.getcwd(), '..')) # Update project_root
        else:
            print(f"Warning: .env file not found at explicit path '{dotenv_path}', nor in CWD or parent of CWD.")
            # As a last resort, if the notebook is in a subfolder of the known root:
            if notebook_dir.startswith(project_root_candidate) and notebook_dir != project_root_candidate:
                 dotenv_path = os.path.join(project_root_candidate, '.env') # Re-assert known root
            # else: project_root_candidate remains the explicit path for saving files later

# Use the determined project_root_candidate for saving files later
project_root = project_root_candidate
print(f"Project root identified as: {project_root}")
print(f"Attempting to load .env file from: {dotenv_path}")

if os.path.exists(dotenv_path) and load_dotenv(dotenv_path=dotenv_path):
    print(f"Successfully loaded .env file from: {dotenv_path}")
else:
    print(f"Failed to load .env file from {dotenv_path}. Trying default load_dotenv().")
    if load_dotenv(): # Tries to find .env in CWD or parent if path not specified
        print("Successfully loaded .env file using default search.")
        # If default search worked, .env might be in CWD. Update project_root if needed for consistency.
        if os.path.exists(os.path.join(os.getcwd(), '.env')):
            project_root = os.getcwd()
            print(f"Updated project root to current working directory: {project_root}")
    else:
        print("Failed to load .env file using default search as well.")
        print(f"Please ensure '.env' exists at '{os.path.join(project_root, '.env')}' or in your current working directory, and contains your credentials.")


CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')

sp = None # Initialize sp
if CLIENT_ID and CLIENT_SECRET:
    try:
        client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
        print("Successfully authenticated with Spotify API!")
    except Exception as e:
        print(f"Error during authentication: {e}")
else:
    print("Error: SPOTIPY_CLIENT_ID or SPOTIPY_CLIENT_SECRET not found in environment variables.")
    print("Please check your .env file loading and its content.")

Project root identified as: c:\Users\balde\SongRecommendation
Attempting to load .env file from: c:\Users\balde\SongRecommendation\.env
Successfully loaded .env file from: c:\Users\balde\SongRecommendation\.env
Successfully authenticated with Spotify API!
